In [1]:
import pandas  as pd
import camelot as cm

from PyPDF2 import PdfReader, PdfWriter

from pathlib import Path
import os 


# Quiero traer el archivo a extraer

pdf_path = Path("C:/Users/yamak/Desktop/Portafolio/Proyecto 01/Finanzas personales/PDF Fix/04-Abril.pdf")

# Leer el archivo PDF
# Extrae bien la informacion con stream, en vez de lattice
tablas = cm.read_pdf(str(pdf_path), pages='all', flavor='stream')
print(f'Se encontraron {len(tablas)} tablas en el PDF.')
# Mostrar la primera tabla como ejemplo
print(tablas[1].df.head())
# Identifico cuantas posibles tablas hay por pdf
tablas[0].to_csv('tabla_0.csv')

# Se identifica que por extracto hay mas de una tabla, se procede a extraer todas las tablas.

Se encontraron 2 tablas en el PDF.
     0           1           2                                 3          4 5  \
0  453  13-04-2025  14-04-2025                  IMPUESTO DECRETO      11.00     
1  454  13-04-2025  14-04-2025  COMPRA POS ASC OXXO SANTA MATILD   2,700.00     
2  455  13-04-2025  14-04-2025                  IMPUESTO DECRETO     240.00     
3  456  13-04-2025  14-04-2025  COMPRA POS ASC WL *STEAM PURCHAS  59,999.00     
4  457  15-04-2025  15-04-2025                  IMPUESTO DECRETO     155.00     

              6  
0  7,293,875.20  
1  7,291,175.20  
2  7,290,935.20  
3  7,230,936.20  
4  7,230,781.20  


In [ ]:
#listo mis pdfs en una variable tipo lista
# me aseguro de que solo sean archivos pdf
pdf_files =  list(Path("C:/Users/yamak/Desktop/Portafolio/Proyecto 01/Finanzas personales/PDF Listos").glob("*.pdf"))
#print(pdf_files)
all_tables = []

for pdf_file in pdf_files:
    # Creo mi variable tabla y extraigo todas las tablas del pdf
    tablas = cm.read_pdf(str(pdf_file), pages='all', flavor='stream')
    for tabla in tablas:
        ncols = len(tabla.df.columns)
        #agrego una columna con el nombre del pdf
        tabla.df['source_pdf'] = pdf_file.name 
        print(f"Antes del if ->{tabla.df.columns} y tiene {ncols} columnas.")
        # Verifico que la tabla tenga 8 columnas
        if ncols == 7:
            #agrego la tabla al listado de todas las tablas
            all_tables.append(tabla.df)
            print(f"La tabla del archivo {pdf_file.name} tiene 7 columnas y se agregará.")
        if ncols == 6:
            #agrego una columna vacia para que tenga 8 columnas
            tabla.df.rename(columns={5:6}, inplace=True)
            tabla.df.insert(5, 5, '')
            all_tables.append(tabla.df)
        if ncols < 6:
            # no la agrego a la lista
            print(f"La tabla del archivo {pdf_file.name} tiene menos de 6 columnas y no se agregará.")
            print(len(tabla.df.columns))

    
# Concateno todas las tablas en un solo DataFrame
df = pd.concat(all_tables, ignore_index=True)
#print(df.head())

#all_tables[1].to_csv('CSV/tabla_ejemplo2.csv', index=False)
df.to_csv("CSV/todas_las_tablas.csv", index=False)

Antes del if ->Index([0, 1, 2, 3, 4, 5, 6, 'source_pdf'], dtype='object') y tiene 7 columnas.
La tabla del archivo 01-Enero.pdf tiene 7 columnas y se agregará.
Antes del if ->Index([0, 1, 2, 3, 4, 5, 6, 'source_pdf'], dtype='object') y tiene 7 columnas.
La tabla del archivo 01-Enero.pdf tiene 7 columnas y se agregará.
Antes del if ->Index([0, 1, 2, 3, 4, 5, 'source_pdf'], dtype='object') y tiene 6 columnas.
Antes del if ->Index([0, 1, 2, 3, 4, 5, 6, 'source_pdf'], dtype='object') y tiene 7 columnas.
La tabla del archivo 02-Febrero.pdf tiene 7 columnas y se agregará.
Antes del if ->Index([0, 1, 2, 3, 4, 5, 'source_pdf'], dtype='object') y tiene 6 columnas.
Antes del if ->Index([0, 1, 2, 3, 4, 5, 6, 'source_pdf'], dtype='object') y tiene 7 columnas.
La tabla del archivo 03-Marzo.pdf tiene 7 columnas y se agregará.
Antes del if ->Index([0, 1, 2, 3, 4, 5, 'source_pdf'], dtype='object') y tiene 6 columnas.
Antes del if ->Index([0, 1, 2, 3, 4, 5, 6, 'source_pdf'], dtype='object') y tiene 7 c

OSError: Cannot save file into a non-existent directory: 'CSV'

In [ ]:
for tablas in all_tables:
    print(tablas.columns)


In [ ]:
df_todos = pd.read_csv('CSV/todas_las_tablas.csv')
#df_todos.info()

# Limpieza de datos basica
# Renombro las columnas
df_todos.columns = ['ID', 'Fecha transaccion', 'Fecha reflejado', 'Descripcion', 'Cargos', 'Abonos', 'Saldo', 'source_pdf']
#df_todos.info()

df_todos['Cargos'] = df_todos['Cargos'].astype(str).str.replace(',', '').astype(float)
df_todos['Abonos'] = df_todos['Abonos'].astype(str).str.replace(',', '').astype(float)
df_todos['Saldo'] = df_todos['Saldo'].astype(str).str.replace(',', '').astype(float)

print(df_todos.info())


#df_todos.to_csv('CSV/todas_las_tablas_limpias.csv', index=False)

In [ ]:
reglas = {
    'impuesto': 'Impuesto',
    'dlo*didi': 'Didi',
    'rappi': 'Rappi',
    'restaurante': 'Restaurante',
    'retiro': 'Retiro Cajero',
    'crepesywaffle': 'Crepes y Waffle',
    'emilio zea': 'Arriendo',
    'arbol de vida': 'Psicologo',
    'aliexpress': 'Aliexpress',
    'arturo calle': 'Arturo Calle',
    'auhmado': 'Ahumado',
    'radiologico': 'Radiologo',
    'raviolera': 'Raviolera',
    'tea for u': 'Tea for U',
    'brahama': 'Brahama',
    'brasas y pollos': 'Brasas y Pollos',
    'brewpub': 'Brewpub',
    'farmacia': 'Farmacia',
    'longo': 'Longo',
    'corral': 'Corral',
    'cruz verde': 'Cruz Verde',
    'decathlon': 'Decathlon',
    'google': 'Servicio Google',
    'drogas': 'Drogeria',
    'drogueria': 'Drogeria',
    'el carnal': 'El carnal',
    'frattosaverio': 'Frattosaverio',
    'frisby': 'Frisby',
    'frutitos': 'Ensalada de fruta',
    'fullcarga colombi': 'Fullcarga Colombia',
    'fundacion mulato': 'Fundacion Mulato',
    'hamburguesas el c': 'Corral',
    'homecenter': 'Homecenter',
    'ilforno': 'Ilforno',
    'juan valdez': 'Juan Valdez',
    'kfc': 'KFC',
    'ko asian kitchen': 'KO',
    'libreria': 'Libros',
    'mpo*mercado pago': 'Mercado Libre',
    'montolivo pasta': 'Montolivo',
    'newrest catering': 'Newrest catering',
    'opticas gmo': 'Gafas',
    'oxxo': 'OXXO',
    'panaderia y paste': 'Panaderia',
    'panamericana': 'Panamericana',
    'payu*robinfood': 'Robinfood',
    'restaurante gratin': 'Gratin Burger',
    'restaurante wok': 'WOK',
    'sandwich': 'Sandwich',
    'taxis libres': 'Taxi',
    'cafe juan': 'Juan Valdez',
    'tienda d1': 'D1',
    'totto': 'Totto',
    'almacenes atenas': 'Almacenes Atenas',
    'av dorado premium': 'AV Dorado Premium',
    'bogota beer compa': 'Bogota Beer',
    'consultorio odont': 'Odontologo',
    'diluca': 'Di luca',
    'dorotea parrilla': 'Dorotea parrilla',
    'dunkin donuts': 'Dunkin Donuts',
    'fit fast food': 'Fit Fast Food',
    'home burgers': 'Home Burger',
    'la lucha sanguche': 'La sangucheria',
    'makro': 'Makro',
    'matuka': 'Matuka',
    'novaventa': 'Novaventa',
    'restaurante la fe': 'Restaurante la fe',
    'salchimonster': 'Salchimonster',
    'santiario': 'Santurario',
    'sr bono': 'Sr Bono',
    'subway': 'Subway',
    'oma': 'Oma',
    'tostao': 'Tostao',
    'turmeque santa ba': 'Turmeque Santa Bar',
    'seguro de hurto': 'Seguro Hurto'
}

def clasificar_transaccion(texto):
    texto = str(texto).lower()
    for palabra, categoria in reglas.items():
        if palabra in texto:
            return categoria
    return 'Otro'  # si no coincide nada

df_todos['Descripcion detalle'] = df_todos['Descripcion'].apply(clasificar_transaccion)


#df_todos.to_csv('CSV/todas_las_tablas_limpias_v22.csv', index=False)

In [ ]:
reglas = {
    'impuesto': 'Impuesto',
    'dlo*didi': 'Aplicaciones',
    'rappi': 'Aplicaciones',
    'restaurante': 'Comida',
    'retiro': 'Retiro Cajero',
    'crepesywaffle': 'Comida',
    'emilio zea': 'Arriendo',
    'arbol de vida': 'Psicologo',
    'aliexpress': 'Aplicaciones',
    'arturo calle': 'Ropa',
    'auhmado': 'Comida',
    'radiologico': 'Radiologo',
    'raviolera': 'Comida',
    'tea for u': 'Comida',
    'brahama': 'Ropa',
    'brasas y pollos': 'Comida',
    'brewpub': 'Comida',
    'farmacia': 'Farmacia',
    'longo': 'Comida',
    'corral': 'Comida',
    'cruz verde': 'Farmacia',
    'decathlon': 'Ropa',
    'google': 'Servicio Google',
    'drogas': 'Farmacia',
    'drogueria': 'Farmacia',
    'el carnal': 'Comida',
    'frattosaverio': 'Comida',
    'frisby': 'Comida',
    'frutitos': 'Comida',
    'fullcarga colombi': 'Otro',
    'fundacion mulato': 'Otro',
    'hamburguesas el c': 'Comida',
    'homecenter': 'Aseo y hogar',
    'ilforno': 'Comida',
    'juan valdez': 'Cafe',
    'kfc': 'Comida',
    'ko asian kitchen': 'Comida',
    'libreria': 'Libros',
    'mpo*mercado pago': 'Mercado Libre',
    'montolivo pasta': 'Comida',
    'newrest catering': 'Comida',
    'opticas gmo': 'Gafas',
    'oxxo': 'Comida',
    'panaderia y paste': 'Comida',
    'panamericana': 'Panamericana',
    'payu*robinfood': 'Comida',
    'restaurante gratin': 'Comida',
    'restaurante wok': 'Comida',
    'sandwich': 'Comida',
    'taxis libres': 'Transporte',
    'cafe juan': 'Cafe',
    'tienda d1': 'Mercado',
    'totto': 'Ropa',
    'almacenes atenas': 'Otro',
    'av dorado premium': 'Otro',
    'bogota beer compa': 'Recreacion',
    'consultorio odont': 'Odontologo',
    'diluca': 'Comida',
    'dorotea parrilla': 'Comida',
    'dunkin donuts': 'Comida',
    'fit fast food': 'Comida',
    'home burgers': 'Comida',
    'la lucha sanguche': 'Comida',
    'makro': 'Aseo y hogar',
    'matuka': 'Comida',
    'novaventa': 'Otro',
    'restaurante la fe': 'Comida',
    'salchimonster': 'Comida',
    'santiario': 'Comida',
    'sr bono': 'Comida',
    'subway': 'Comida',
    'oma': 'Cafe',
    'tostao': 'Cafe',
    'turmeque santa ba': 'Recreacion',
    'seguro de hurto': 'Seguro'
}

def clasificar_transaccion(texto):
    texto = str(texto).lower()
    for palabra, categoria in reglas.items():
        if palabra in texto:
            return categoria
    return 'Otro'  # si no coincide nada

df_todos['Categoria movimiento'] = df_todos['Descripcion'].apply(clasificar_transaccion)
df_todos.to_csv('CSV/todas_las_tablas_limpias_v2.csv', index=False)